In [142]:
import pandas as pd
import numpy as np

import bamboolib as bam

In [143]:
URL = 'https://raw.githubusercontent.com/globaldothealth/monkeypox/main/latest.csv'

data = pd.read_csv(URL)

data

,ID,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,...,Travel_history_country,Genomics_Metadata,Confirmation_method,Source,Source_II,Date_entry,Date_last_modified,Source_III,Source_IV,Country_ISO3
0,1,confirmed,Guy's and St Thomas Hospital London,London,England,NaN,NaN,2022-04-29,2022-05-06,rash,...,Nigeria,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,https://www.who.int/emergencies/disease-outbre...,2022-05-18,2022-05-18,NaN,NaN,GBR
1,2,confirmed,Guy's and St Thomas Hospital London,London,England,NaN,NaN,2022-05-05,2022-05-12,rash,...,NaN,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,NaN,2022-05-18,2022-05-18,NaN,NaN,GBR
2,3,confirmed,London,London,England,NaN,NaN,2022-04-30,2022-05-13,vesicular rash,...,NaN,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,NaN,2022-05-18,2022-05-18,NaN,NaN,GBR
3,4,confirmed,London,London,England,NaN,male,NaN,2022-05-15,vesicular rash,...,NaN,West African Clade,NaN,https://www.gov.uk/government/news/monkeypox-c...,NaN,2022-05-18,2022-05-18,NaN,NaN,GBR
4,5,confirmed,London,London,England,NaN,male,NaN,2022-05-15,vesicular rash,...,NaN,West African Clade,NaN,https://www.gov.uk/government/news/monkeypox-c...,NaN,2022-05-18,2022-05-18,NaN,NaN,GBR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3564,3565,confirmed,Texas,NaN,United States,NaN,NaN,NaN,2022-06-21,NaN,...,NaN,NaN,NaN,https://www.cdc.gov/poxvirus/monkeypox/respons...,NaN,2022-06-21,2022-06-21,NaN,NaN,USA
3565,3566,confirmed,Utah,NaN,United States,NaN,NaN,NaN,2022-06-21,NaN,...,NaN,NaN,NaN,https://www.cdc.gov/poxvirus/monkeypox/respons...,NaN,2022-06-21,2022-06-21,NaN,NaN,USA
3566,3567,confirmed,Washington,NaN,United States,NaN,NaN,NaN,2022-06-21,NaN,...,NaN,NaN,NaN,https://www.cdc.gov/poxvirus/monkeypox/respons...,NaN,2022-06-21,2022-06-21,NaN,NaN,USA
3567,3568,confirmed,Indiana,NaN,United States,NaN,NaN,NaN,2022-06-21,NaN,...,NaN,NaN,NaN,https://www.cdc.gov/poxvirus/monkeypox/respons...,NaN,2022-06-21,2022-06-21,NaN,NaN,USA


In [144]:
data_pox =data[data.columns[data.isna().sum()/data.shape[0]<0.85]]
data_pox = data_pox.loc[~(data_pox['Status'].isin(['discarded']))]
data_pox = data_pox.drop(columns=['Location', 'City', 'Gender', 'Source_II', 'ID', 'Source','Date_last_modified'])

suspected_data_pox = data_pox.loc[data_pox['Status'].isin(['suspected'])]
suspected_data_pox['date'] = pd.to_datetime(suspected_data_pox['Date_entry'], format='%Y-%m-%d')
confirmed_data_pox = suspected_data_pox.groupby(['Country', 'date']).agg(suspected=('Status', 'size')).reset_index()


confirmed_data_pox = data_pox.loc[data_pox['Status'].isin(['confirmed'])]
confirmed_data_pox['date'] = pd.to_datetime(confirmed_data_pox['Date_confirmation'], format='%Y-%m-%d')
confirmed_data_pox = confirmed_data_pox.groupby(['Country', 'date']).agg(confirmed=('Status', 'size')).reset_index()
#confirmed_data_pox['confirmed'] = confirmed_data_pox.groupby(['Country'])['daily_cases'].cumsum()
#confirmed_data_pox = confirmed_data_pox.sort_values(by=['date'], ascending = True)

In [145]:
confirmed_data_pox.Country.unique()

array(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Canada',
       'Chile', 'Czech Republic', 'Denmark', 'England', 'Finland',
       'France', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece',
       'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Latvia',
       'Lebanon', 'Luxembourg', 'Malta', 'Mexico', 'Morocco',
       'Netherlands', 'Northern Ireland', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Scotland', 'Serbia', 'Singapore', 'Slovenia', 'Spain',
       'Sweden', 'Switzerland', 'United Arab Emirates', 'United States',
       'Venezuela', 'Wales'], dtype=object)

In [146]:
len(confirmed_data_pox.Country.unique())

45

In [147]:
print(confirmed_data_pox.Country.unique())

['Argentina' 'Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile'
 'Czech Republic' 'Denmark' 'England' 'Finland' 'France' 'Georgia'
 'Germany' 'Ghana' 'Gibraltar' 'Greece' 'Hungary' 'Iceland' 'Ireland'
 'Israel' 'Italy' 'Latvia' 'Lebanon' 'Luxembourg' 'Malta' 'Mexico'
 'Morocco' 'Netherlands' 'Northern Ireland' 'Norway' 'Poland' 'Portugal'
 'Romania' 'Scotland' 'Serbia' 'Singapore' 'Slovenia' 'Spain' 'Sweden'
 'Switzerland' 'United Arab Emirates' 'United States' 'Venezuela' 'Wales']


In [148]:
from datetime import date


def map_dateRange(df, w_countries):
    date_rng = pd.date_range(start='5/1/2022', end=confirmed_data_pox['date'].max())
    df_date = pd.DataFrame(date_rng,  columns=['date'])
    df = df[df['Country']==w_countries]
    df = df.merge(df_date, how='outer', on = ['date'])
    df['Country'] = df['Country'].fillna(w_countries)
    df['confirmed']=df['confirmed'].fillna(0)
    df = df.sort_values(by=['date'],  ascending=True, axis=0)
    return df

In [149]:
argentina = map_dateRange(confirmed_data_pox,'Argentina')
australia = map_dateRange(confirmed_data_pox,'Australia')
austria = map_dateRange(confirmed_data_pox,'Austria')
belgium = map_dateRange(confirmed_data_pox,'Belgium')
brazil = map_dateRange(confirmed_data_pox,'Brazil')
canada = map_dateRange(confirmed_data_pox,'Canada')
czechrepublic = map_dateRange(confirmed_data_pox,'Czech Republic')
Chile =  map_dateRange(confirmed_data_pox, 'Chile')
Denmark = map_dateRange(confirmed_data_pox,'Denmark')
England = map_dateRange(confirmed_data_pox,'England')
Finland = map_dateRange(confirmed_data_pox,'Finland')
France = map_dateRange(confirmed_data_pox,'France')
Germany = map_dateRange(confirmed_data_pox,'Germany')
Georgia = map_dateRange(confirmed_data_pox,'Georgia')
Ghana = map_dateRange(confirmed_data_pox,'Ghana')
Gibraltar = map_dateRange(confirmed_data_pox,'Gibraltar')
Greece = map_dateRange(confirmed_data_pox,'Greece')
Hungary = map_dateRange(confirmed_data_pox,'Hungary')
Iceland = map_dateRange(confirmed_data_pox,'Iceland')
Ireland = map_dateRange(confirmed_data_pox,'Ireland')
Israel = map_dateRange(confirmed_data_pox,'Israel')
Italy = map_dateRange(confirmed_data_pox,'Italy')
Latvia = map_dateRange(confirmed_data_pox,'Latvia')
Lebanon = map_dateRange(confirmed_data_pox,'Lebanon')
Luxembourg = map_dateRange(confirmed_data_pox,'Luxembourg')
Mexico = map_dateRange(confirmed_data_pox,'Mexico')
Malta = map_dateRange(confirmed_data_pox,'Malta')
Morocco = map_dateRange(confirmed_data_pox,'Morocco')
Netherlands = map_dateRange(confirmed_data_pox,'Netherlands')
NorthernIreland = map_dateRange(confirmed_data_pox,'Northern Ireland')
Norway = map_dateRange(confirmed_data_pox,'Norway')
Poland = map_dateRange(confirmed_data_pox,'Poland')
Portugal = map_dateRange(confirmed_data_pox,'Portugal')
Romania = map_dateRange(confirmed_data_pox,'Romania')
Scotland = map_dateRange(confirmed_data_pox,'Scotland')
Serbia = map_dateRange(confirmed_data_pox,'Serbia')
Slovenia = map_dateRange(confirmed_data_pox,'Slovenia')
Spain = map_dateRange(confirmed_data_pox,'Spain')
Sweden = map_dateRange(confirmed_data_pox,'Sweden')
Switzerland = map_dateRange(confirmed_data_pox,'Switzerland')
UnitedArabEmirates = map_dateRange(confirmed_data_pox,'United Arab Emirates')
UnitedStates = map_dateRange(confirmed_data_pox,'United States')
Venezuela = map_dateRange(confirmed_data_pox,'Venezuela')
Wales = map_dateRange(confirmed_data_pox,'Wales')







In [150]:
df_monkeypox = pd.concat([argentina, australia, austria, 
                          belgium, brazil, canada, czechrepublic, 
                          Denmark, England, Finland, France, Germany, 
                          Ghana, Gibraltar, Greece, Hungary, Ireland, Israel, 
                          Italy, Latvia, Mexico, Malta, Morocco, Netherlands, 
                          NorthernIreland, Norway, Poland, Portugal, Scotland, Slovenia, 
                          Spain, Sweden, Switzerland, UnitedArabEmirates, 
                          UnitedStates, Wales, Georgia, Chile, Iceland, Lebanon, Luxembourg, Venezuela, Romania, Serbia], axis=0, ignore_index=True)


In [151]:
df_monkeypox['acc_confirmed'] = df_monkeypox.groupby(['Country'])['confirmed'].cumsum()
df_monkeypox['acc_confirmed'] = pd.to_numeric(df_monkeypox['acc_confirmed'], downcast='integer', errors='coerce')
df_monkeypox['confirmed'] = pd.to_numeric(df_monkeypox['confirmed'], downcast='integer', errors='coerce')


In [152]:
df_world = df_monkeypox.groupby(['date']).agg(acc_confirmed_sum=('acc_confirmed', 'sum'), confirmed_sum=('confirmed', 'sum')).reset_index()
df_world['World'] = 'World'

In [153]:
df_world = df_world[['World'] + ['date', 'acc_confirmed_sum', 'confirmed_sum']]
df_world['confirmed_sum'] = pd.to_numeric(df_world['confirmed_sum'], downcast='integer', errors='coerce')
df_world = df_world[['World', 'date', 'confirmed_sum'] + ['acc_confirmed_sum']]
df_world = df_world.rename(columns={'confirmed_sum': 'confirmed', 'acc_confirmed_sum': 'acc_confirmed', 'World': 'Country'})
df_monkeypox = pd.concat([df_world, df_monkeypox], axis=0, ignore_index=True)
df_monkeypox

,Country,date,confirmed,acc_confirmed
0,World,2022-05-01,0,0
1,World,2022-05-02,0,0
2,World,2022-05-03,0,0
3,World,2022-05-04,0,0
4,World,2022-05-05,0,0
...,...,...,...,...
2335,Serbia,2022-06-17,1,1
2336,Serbia,2022-06-18,0,1
2337,Serbia,2022-06-19,0,1
2338,Serbia,2022-06-20,0,1


In [154]:
#get world count
world_count = df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-1]
world_count

3156

In [155]:
#set update date

df_monkeypox['date'].iloc[-1]

Timestamp('2022-06-21 00:00:00')

In [156]:
df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-1]-df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-2]



271

In [157]:
geo = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')


In [158]:
df_monkeypox_uk = df_monkeypox.loc[df_monkeypox['Country'].isin(['England', 'Northern Ireland', 'Scotland', 'Wales'])]


In [159]:
df_monkeypox_word = df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]


In [160]:
df_monkeypox = df_monkeypox.loc[~(df_monkeypox['Country'].isin(['World', 'England', 'Northern Ireland', 'Scotland', 'Wales']))]


In [161]:
df_monkeypox_uk['latitude'] = 55.378051
df_monkeypox_uk['longitude'] = -3.435973

In [162]:
df_monkeypox = pd.merge(df_monkeypox, geo[['country', 'latitude', 'longitude']], how='inner', left_on=['Country'], right_on=['country'])
df_monkeypox = df_monkeypox.reset_index()


In [163]:
df_monkeypox = pd.concat([df_monkeypox, df_monkeypox_uk], axis=0, ignore_index=True)
df_monkeypox = df_monkeypox.drop(columns=['index', 'country'])


In [164]:
df_monkeypox_word['latitude']=0
df_monkeypox_word['longitude']=0

In [165]:
df_monkeypox = pd.concat([df_monkeypox, df_monkeypox_word], axis=0, ignore_index=True)

In [166]:
df_monkeypox = df_monkeypox.rename(columns={'confirmed': 'new_daily_cases'})

In [167]:
df_monkeypox.to_csv('df_monkeyPoxTimeSerie.csv', index_label=False)

In [168]:
#today world cases
df_monkeypox[df_monkeypox['Country']=='World']['acc_confirmed'].max()

3156

In [169]:
df_monkeypox
print(df_monkeypox.Country.unique())

['Argentina' 'Australia' 'Austria' 'Belgium' 'Brazil' 'Canada'
 'Czech Republic' 'Denmark' 'Finland' 'France' 'Germany' 'Ghana'
 'Gibraltar' 'Greece' 'Hungary' 'Ireland' 'Israel' 'Italy' 'Latvia'
 'Mexico' 'Malta' 'Morocco' 'Netherlands' 'Norway' 'Poland' 'Portugal'
 'Slovenia' 'Spain' 'Sweden' 'Switzerland' 'United Arab Emirates'
 'United States' 'Georgia' 'Chile' 'Iceland' 'Lebanon' 'Luxembourg'
 'Venezuela' 'Romania' 'Serbia' 'England' 'Northern Ireland' 'Scotland'
 'Wales' 'World']


In [170]:
f"{world_count:,.0f}"

'3,156'

In [171]:
print(df_monkeypox[df_monkeypox['Country']=='World']['new_daily_cases'].iloc[-1]) 
print(df_monkeypox[df_monkeypox['Country']=='World']['new_daily_cases'].iloc[-2])

271
289


In [172]:
#set data for map

#df_monkeypox.groupby(['Country', 'latitude', 'longitude'])['acc_confirmed'].max().reset_index()

In [173]:
# Create dictionary of list
#testlist = df_monkeypox[['Country', 'latitude', 'longitude']]
#dict_of_locations = testlist.set_index('Country')[['latitude', 'longitude']].T.to_dict('dict')

In [174]:
#dict_of_locations